In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split
import torchvision.models as models

# Impostare il dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preprocessing
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

# Trasformazioni per il training (con data augmentation)
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Flip orizzontale casuale
    transforms.RandomCrop(32, padding=4),  # Crop casuale con padding
    transforms.ToTensor(),  # Converti in tensore
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizzazione
])

# Trasformazioni per validazione e test (senza data augmentation)
val_test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset per training e test
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True)

# Applica trasformazioni diverse per training e validazione
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=train_transform, download=True)
val_dataset = datasets.CIFAR10(root='./data', train=True, transform=val_test_transform, download=True)

# Applica trasformazione per il test
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=val_test_transform, download=True)

# Divisione del training set in training (80%) e validazione (20%)
train_size = int(0.8 * len(train_dataset))  # 80% per il training
val_size = len(full_train_dataset) - train_size  # 20% per la validazione
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# DataLoader per batch processing
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)  # Training con shuffle
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)  # Validazione senza shuffle
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)  # Test senza shuffle

# Stampiamo il numero di esempi
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")


# Caricare ResNet34 pre-addestrata
model = models.resnet34(weights='DEFAULT')

# Modificare l'ultimo strato per CIFAR-10
model.fc = nn.Linear(model.fc.in_features, 10)
# Modifica dell'ultimo fully connected con Dropout
model.fc = nn.Sequential(
    nn.Dropout(0.3),  # Dropout aggiunto
    nn.Linear(model.fc.in_features, 10)
)
model = model.to(device)

# Loss e ottimizzatore
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4) 
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

# Funzioni di addestramento e validazione
def train(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    return total_loss / len(loader), 100. * correct / total


def validate(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return total_loss / len(loader), 100. * correct / total


# Ciclo di addestramento
num_epochs = 50
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = validate(model, val_loader, criterion)
    scheduler.step()

    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

# Testare il modello
test_loss, test_acc = validate(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Training set size: 40000
Validation set size: 10000
Test set size: 10000
Epoch 1/50:
Train Loss: 2.6856, Train Acc: 14.93%
Val Loss: 2.2810, Val Acc: 16.89%
Epoch 2/50:
Train Loss: 2.1420, Train Acc: 22.68%
Val Loss: 1.9152, Val Acc: 27.14%
Epoch 3/50:
Train Loss: 1.7889, Train Acc: 31.04%
Val Loss: 1.8623, Val Acc: 32.24%
Epoch 4/50:
Train Loss: 1.6726, Train Acc: 37.39%
Val Loss: 1.6193, Val Acc: 40.21%
Epoch 5/50:
Train Loss: 1.5631, Train Acc: 42.07%
Val Loss: 1.6280, Val Acc: 39.43%


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split
import torchvision.models as models

# Impostare il dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preprocessing con aumentata data augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Dataset e split
dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

# Caricare ResNet34 pre-addestrata
model = models.resnet34(pretrained=True)

# Modifica dell'ultimo fully connected con Dropout
model.fc = nn.Sequential(
    nn.Dropout(0.5),  # Dropout aggiunto
    nn.Linear(model.fc.in_features, 10)
)

model = model.to(device)

# Loss e ottimizzatore
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # Aumentato il weight_decay

# Funzioni di addestramento e validazione
def train(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    return total_loss / len(loader), 100. * correct / total


def validate(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return total_loss / len(loader), 100. * correct / total


# Ciclo di addestramento
num_epochs = 50
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = validate(model, val_loader, criterion)

    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

# Testare il modello
test_loss, test_acc = validate(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/50:
Train Loss: 1.3868, Train Acc: 51.60%
Val Loss: 1.1498, Val Acc: 59.83%
Epoch 2/50:
Train Loss: 0.9809, Train Acc: 66.63%
Val Loss: 1.4209, Val Acc: 52.86%
Epoch 3/50:
Train Loss: 1.0299, Train Acc: 65.32%
Val Loss: 0.9120, Val Acc: 68.76%
Epoch 4/50:
Train Loss: 0.8262, Train Acc: 71.94%
Val Loss: 0.8041, Val Acc: 72.16%
Epoch 5/50:
Train Loss: 0.7486, Train Acc: 74.36%
Val Loss: 0.7989, Val Acc: 72.23%
Epoch 6/50:
Train Loss: 0.7715, Train Acc: 73.79%
Val Loss: 0.9850, Val Acc: 67.95%


KeyboardInterrupt: 